In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/

It would be bit cleaner to use nltk to do the tokenization, but we don't have nltk installed in our cluster.

In [ ]:
def tokenize(s):
    import re
    stopwords = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'])
    word_regex = '^[a-z][a-z\'-]+[a-z]$'
    s = s.lower()
    arr = s.split()
    terms = []
    for term in arr:
        if re.match(word_regex, term) != None and len(term) > 3 and term not in stopwords:
            terms.append(term)
    return terms


In [ ]:
test_strings = ['the quick brown fox jumps over the brown fence.',
              'the boy paints a tall fence brown!',
              'basketball players are tall.',
              'quick basketball players jump high']

In [ ]:
tokens = sc.parallelize(test_strings).map(tokenize)

In [ ]:
tokens.collect()

In [ ]:
vocab = tokens.flatMap(lambda words: words).distinct()
vocab.collect()

In [ ]:
from collections import Counter
import numpy as np

#sc.broadcast shares an immutable object throughout the cluster
broadcastVocab = sc.broadcast(vocab.collect())

def bow_vectorize(tokens):
    word_counts = Counter(tokens)
    vector = [word_counts[v] if v in word_counts else 0 for v in broadcastVocab.value]
    return np.array(vector)

In [ ]:
tokens.map(bow_vectorize).collect()

In [ ]:
broadcastVocab.value

In [ ]:
term_freq = tokens.map(lambda terms: Counter(terms))

In [ ]:
doc_freq = term_freq.flatMap(lambda counts: counts.keys()).map(lambda keys: (keys, 1)).reduceByKey(lambda a, b: a + b)
doc_freq.collect()

In [ ]:
total_docs = term_freq.count()
total_docs

In [ ]:
import math

idf = doc_freq.map(lambda tup: (tup[0], math.log(float(total_docs)/ (1 + tup[1])))).collect()
idf

In [ ]:
broadcast_idf = sc.broadcast(idf)

In [ ]:
def tfidf_vectorize(tokens):
    word_counts = Counter(tokens)
    doc_length = sum(word_counts.values())
    
    vector = [ word_counts.get(word[0], 0) * word[1] / float(doc_length) for word in broadcast_idf.value ]
    return np.array(vector)

In [ ]:
tfidf = tokens.map(tfidf_vectorize)
tfidf.collect()

In [ ]:
bow = tokens.map(bow_vectorize).cache()
bow.collect()

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from math import sqrt

In [ ]:
clusters = KMeans.train(tfidf, 2, maxIterations=10, initializationMode="random")

In [ ]:
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
WSSSE = tfidf.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

In [ ]:
clusters.centers

In [ ]:
top_n = 3
print([idf[idx][0] for idx in [np.argsort(x)[::-1][:top_n] for x in clusters.centers][0]])
print([idf[idx][0] for idx in [np.argsort(x)[::-1][:top_n] for x in clusters.centers][1]])

In [ ]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

In [ ]:
!wget https://dsr-data.s3.amazonaws.com/enron/enron.json

In [ ]:
# read as dataframe 
email = sqlCtx.read.json('enron.json')

In [ ]:
email.take(1)

In [ ]:
# tokenize documents
tokenized_rdd = email.select('text').rdd \
  .map(lambda row: row.text) \
  .map(lambda text: text.replace('\n', ' ').replace('\r', ' ')) \
  .map(lambda text: tokenize(text)) \
  .filter(lambda text: len(text) > 0)

# compute term and document frequencies
term_frequency = tokenized_rdd.map(lambda terms: Counter(terms))

doc_frequency = term_frequency.flatMap(lambda counts: counts.keys()) \
                             .map(lambda keys: (keys, 1)) \
                             .reduceByKey(lambda a, b: a + b)

In [ ]:
sc.stop()